# Capstone Project - The Battle of Neighborhoods (Week 1)

## Upload Libraries Required

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 1. Introduction Section :
### Discussion of the business problem and the audience who would be interested in this project.
### Description of the Problem and Background
Scenario:
I am a Business Analyst in Middle of Jakarta. I currently live within walking distance to Downtown Dukuh Atas MRT metro station and I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment. I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How can I find a convenient and enjoyable place similar to mine now in Singapore? Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:

- apartment must be 2 or 3 bedrooms
- desired location is near a metro station in the Manhattan area and within 1.0 mile (1.6 km) radius
- price of rent not exceed $7,000 per month
- top ammenities in the selected neighborhood shall be similar to current residence
- desirable to have venues such as coffee shops, restaurants Asian Thai, wine stores, gym and food shops
- as a reference, I have included a map of venues near current residence in Jakarta.

### Business Problem:
The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

### Interested Audience
I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

# 2. Data Section:¶
## Description of the data and its sources that will be used to solve the problem
## Description of the Data:¶
The following data is required to answer the issues of the problem:

- List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)
- List of Subway metro stations in Manhattan with their address location
- List of apartments for rent in Manhattan area with their addresses and price
- Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
- Venues for each Manhattan neighborhood ( than can be clustered)
- Venues for subway metro stations, as needed

## How the data will be used to solve the problem
The data will be used as follows:

- Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
- Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
- Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
- create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
- Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
- Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.

The procesing of these DATA will allow to answer the key questions to make a decision:

- what is the cost of rent (per square ft) around a mile radius from each subway metro station?
- what is the area of Manhattan with best rental pricing that meets criteria established?
- What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
- What are the venues of the two best places to live? How the prices compare?
- How venues distribute among Manhattan neighborhoods and around metro stations?
- Are there tradeoffs between size and price and location?
- Any other interesting statistical data findings of the real estate and overall data.

## Reference of venues around current residence in Jakarta for comparison to Manhattan place

In [2]:
address = 'The Capital Residence, Jakarta'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Jakarta home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Jakarta home are -6.2253406, 106.81125594971024.


In [3]:
neighborhood_latitude=-6.2253406
neighborhood_longitude=106.81125594971024

In [5]:
CLIENT_ID = 'EQMYB123SFAJXELDHKOYAGRHGGUZYM3E23ADNCHKFYB0ET41' # your Foursquare ID
CLIENT_SECRET = 'ONJHNFQH1HYMSRUU10WBUAADC14Y0AIKPXHDDHTWUARGUX40' # your Foursquare Secret
VERSION = '20201023'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EQMYB123SFAJXELDHKOYAGRHGGUZYM3E23ADNCHKFYB0ET41
CLIENT_SECRET:ONJHNFQH1HYMSRUU10WBUAADC14Y0AIKPXHDDHTWUARGUX40


In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EQMYB123SFAJXELDHKOYAGRHGGUZYM3E23ADNCHKFYB0ET41&client_secret=ONJHNFQH1HYMSRUU10WBUAADC14Y0AIKPXHDDHTWUARGUX40&v=20201023&ll=-6.2253406,106.81125594971024&radius=500&limit=100'

In [7]:
results = requests.get(url).json()

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:

venues = results['response']['groups'][0]['items']
    
JKTnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
JKTnearby_venues =JKTnearby_venues.loc[:, filtered_columns]

# filter the category for each row
JKTnearby_venues['venue.categories'] = JKTnearby_venues.apply(get_category_type, axis=1)

# clean columns
JKTnearby_venues.columns = [col.split(".")[-1] for col in JKTnearby_venues.columns]

JKTnearby_venues.head(10)

C:\Users\RidwanRamadhan\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Hard Rock Cafe Jakarta,American Restaurant,-6.225280,106.810270
1,The Ritz-Carlton Jakarta Pacific Place,Hotel,-6.224051,106.809820
2,Pacific Place,Shopping Mall,-6.224843,106.809792
3,Kem Chicks,Grocery Store,-6.224707,106.809806
4,THE GOODS DEPT •,Department Store,-6.224947,106.810339
5,Cork&Screw,Wine Bar,-6.225700,106.808919
6,Shaburi,Shabu-Shabu Restaurant,-6.224868,106.809158
7,Tanamera Coffee,Coffee Shop,-6.223594,106.809684
8,GROM - IL GELATO COME UNA VOLTA,Ice Cream Shop,-6.224901,106.809233
9,Gahyo Korean Grill & Cuisine,Korean Restaurant,-6.226630,106.809873


In [10]:
map_JKT = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(JKTnearby_venues['lat'], JKTnearby_venues['lng'], JKTnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_JKT)  
    
map_JKT